In [ ]:
#origin code

# -*- coding: utf-8 -*-
"""
Created on Thu Sep 29 01:42:29 2022

@author: ASUS

исходный файл: эксель со столбцами: 
    PDBid	chain	start*	end*
*-по PDB файлу

аут файл:


Описание функций:
    def Start - указываем путь к файлу xlsx, на выходе считанный датафрейм. [Вход: путь к экселю. Выход: считанный датафрейм]
    def Change_df - (!!! меняет исходный датафрейм) преобразует датафрейм к виду PDBid_chain PDBid chain start end length [Вход: датафрейм. Выход: преобразованный датафрейм. Сохраняет: эксель с преобразованным датафреймом]
    def Hist_maker - строит гистограмму по длинам подборки [Вход: подборка с длинами. Сохраняет: картинку гистограмму распределения длин молекул подборки]
    def Alpha_helix - добывает координаты альфа-спиралей [Вход: подборка наша. Вызывает: list_struc_maker (выход: датафрейм со спиралями их координатами)]
    def list_struc_maker - создаёт список длиной скоько а.о. и заполняет 1 там где есть структура [Вход: датафрейм со спиралями Helixes, длина белка l, начало белка start_domain. Вызывается в Alpha_helix]
    
"""

import numpy as np
import pandas as pd
import time
#import seaborn as sns
#import matplotlib.pyplot as plt
#from tempfile import gettempdir
#import biotite
#import biotite.structure as struc
#import biotite.structure.io.mmtf as mmtf
#import biotite.structure.io.pdb as pdb
import biotite.structure.io.pdbx as pdbx
#import biotite.sequence as seq
#import biotite.sequence.graphics as graphics
import biotite.database.rcsb as rcsb
#import biotite.application.dssp as dssp

#_______________________________________________________

def Start ():
    #-----Формируем первый датафрейм {PDBid chain start end}
    #print('Введите путь к файлу excel с его наименованием и .xlsx')
    #way = str(input())
    #'D:/Оля/курсовая/1_Обработка/1 Работа/проверяю расширенную обработку/pdbID.txt'
    path = 'book.xlsx'
    print("Вы ввели", path) 
    Podborka = pd.read_excel(path) #читвем датафрейм в переменную Podborka
    #print("Original Podborka\n", Podborka.head())
    #print('Типы:', Podborka.dtypes)
    #print('Колонки', Podborka.columns)
    #Podborka['PDBid']=str(Podborka['PDBid'])#преобразуем в строку
    #Podborka['chain']=Podborka['chain'].astype(str)#преобразуем в строку
    #Podborka['PDBid'].astype(str)
    #print('Типы:', Podborka.dtypes)
    print('--------------------')
    return Podborka
    
def Change_df (Podborka):
    #-----Модифицируем датафрейм, добавляя length и PDBid_chain {PDBid_chain PDBid chain start end length}
    Podborka.insert(4, 'length', Podborka['end']-Podborka['start']+1) #считаем длину в а.о. каждого домена/цепи
    Podborka.insert(0, 'PDBid_Chain', Podborka['PDBid']+'_'+Podborka['chain'])#добавим идетификатор PDBid_Chain
    #print("Changed Podborka\n", Podborka.head())
    #Выведем саммари по этому датафрейму
    print('Саммари\n', Podborka.describe())  
    #Podborka.describe().to_csv("Podborka_describe.csv")
    Podborka.describe().to_excel("Podborka_describe.xlsx")#запишем себе данные дискрайба
    #и на всякий случай сохраним датафрейм
    Podborka.to_excel("Podborka.xlsx")
    #Заготовки для результатов по стурктурам
    Podborka["Alpha-helixes"] = np.nan
    Podborka["Beta-sheets"] = np.nan
    Podborka["ThreeTen-helixes"] = np.nan
    Podborka["Count_AlphaH"] = np.nan
    Podborka["Count_BetaS"] = np.nan
    Podborka["Count_ThreeTenH"] = np.nan
    return Podborka
    print('--------------------')

def Hist_maker(Podborka):
    #Сразу распечатаем гистограмму (распределение длин белков подборки)
    ax = Podborka.hist(column='length', bins=25, grid=True, figsize=(12,8), color='royalblue', rwidth=0.9)  
    ax = ax[0]
    for x in ax:
        # Remove title
        x.set_title("Распределение длин белков подборки", size=20)
        # Set x-axis label
        x.set_xlabel("Длина, а.о.", labelpad=20, weight='bold', size=16)
        # Set y-axis label
        x.set_ylabel("Количество, структур", labelpad=20, weight='bold', size=16)
    fig = ax[0].get_figure()
    fig.savefig('Length distribution.jpg')#сохраним гистограмму
    print('--------------------')

def Alpha_helix(Podborka):
    g = 0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о. для PDB обычного
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о. для CATH
            #print("Beg_str=", Beg_str, "type=", type(Beg_str))
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о.
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "1"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_AlphaH"] = Helixes.shape[0]
                #input()
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет альфа-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "альфа-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "альфа-спиралей не обнаружено")
            Podborka.loc[i, "Count_AlphaH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Alpha-helixes"] = vse100
            #print("Podborka", Podborka
        print('--------------------')
    
def ThreeTen_helix(Podborka):
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о.
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о. ЕСЛИ КАК В БД брать нумерацию
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "5"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_ThreeTenH"] = Helixes.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "3_10-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "3_10-спиралей не обнаружено")
            Podborka.loc[i, "Count_ThreeTenH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "ThreeTen-helixes"] = vse100
            #print("Podborka", Podborka)
    print('--------------------')        

def Beta_sheet(Podborka):
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_sheet_range"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_sht=pdbx_file [PDBid, "struct_sheet_range"]["sheet_id"]#id листа
            #print(Id_sht)
            Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_label_seq_id"]#начало листа а.о.
            #Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_seq_id"]#начало листа а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_label_seq_id"]#конец листа а.о.
            #End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_auth_seq_id"]#конец листа а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Chain_sht = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_asym_id"]#цепь https://www.biotite-python.org/examples/gallery/structure/sheet_arrangement.html
            #print("chain_sht", Chain_sht)
            #input()
            #ещё одна проверка если вдруг будет тупняк
            if Id_sht is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Sheeets = pd.DataFrame({'Id_sht' : Id_sht,'Beg_str' : Beg_str, 'End_str': End_str, 'Chain_sht': Chain_sht})
                #print('Sheeets\n', Sheeets)
                #print('type', type(Sheeets))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Sheeets = Sheeets[Sheeets.Chain_sht == chain_tut]#цепь нужная по Podborka
                Sheeets = Sheeets[Sheeets.Beg_str >= start_domain] #входит в домен
                Sheeets = Sheeets[Sheeets.End_str <= stop_domain] #входит в домен
                Sheeets.index = np.arange(len(Sheeets))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Sheeets.index",  Sheeets.index)
                #print("Sheeets которые подходят нам\n", Sheeets)
                Podborka.loc[i, "Count_BetaS"] = Sheeets.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Sheeets.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "бета-листов не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Sheeets, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_sht")
        else:
            print("В белке ", PDBid, "бета-листов не обнаружено")
            Podborka.loc[i, "Count_BetaS"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Beta-sheets"] = vse100
            #print("Podborka", Podborka)
    print('--------------------')        


def Irregular(Podborka):
    for i, row in Podborka.iterrows():
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid просто для отслеживания
        #l = Podborka.iloc[i, 5]#выясняем актуальную длину
        #print("l=", l)
        #print("Podborka.iloc[i, 6]", Podborka.iloc[i, 6])
        list_alpha = Podborka.iloc[i, 6].split(";")#делаем список с данными об альфа
        list_beta = Podborka.iloc[i, 7].split(";")#делаем список с данными о бета-листах
        list_threeten = Podborka.iloc[i, 8].split(";")#делаем список с данными о 310-спиралях
        print("Данные: ", PDBid)
        #print("list_alpha=",list_alpha)
        proba = [0]*100#пустой для наших нерегулярных структур        
        for index in range(len(proba)):
            #print("index=", index, ", list_alpha[index]=", list_alpha[index])
            #print(type(l),"   ", type(float(list_alpha[index])))
            proba[index]=1-float(list_alpha[index])-float(list_beta[index])-float(list_threeten[index])
            proba[index] = round(proba[index], 2)
            if proba[index] < 0:
                print("значение в нерегулярных стурктурах <0!")
                print(PDBid, "index=", index, "proba=", proba)
                #input()
            #print("proba=", proba)
            #vse100 = ';'.join(str(proba))
            vse100 = (";".join(map(str,proba)))
            #print("vse100=", vse100)
            #vse100 = (";", join(map(str, proba)))
            Podborka.loc[i, "Irregular"] = vse100   
    print('--------------------') 
    
            
def list_struc_maker(Helixes, l, start_domain):
    #Считываем нужные альфа-спирали в список 
    list_hel_original=[0] * l#список, где будем выставлять наличие альфа-спиралей
    #заполняем его (это тот, что в аминокислотных остатках)
    for i in range(0,len(Helixes)):
        start = int(Helixes.iloc[i]["Beg_str"])
        stop = int(Helixes.iloc[i]["End_str"])
        #print("СТАРТ=", start, " СТОП=", stop)
        #ставим относительно начала и конца домена с учётом, что в списке начало с 0, а не 1 и учёт домена
        start = start-start_domain
        stop = stop - start_domain
        #print("start=", start, " stop=", stop)
        #вносим в нашу цепь данные
        for c in range (start, stop+1):
            list_hel_original[c]=1
            #print("list_hel_original", list_hel_original[c])
    #print("Итого по цепи")
    #for i in range(0, len(list_hel_original)):
        #print("list_hel_original[",i,"]=", list_hel_original[i])
    Itog = translater(list_hel_original, l)
    #print("itog",Itog)
    return Itog
    print('--------------------')

def translater(list_hel_original, l):
    #делаем список из 100 ячеек (наши координаты, т.е. %)
    proba = [0]*100
    T=l/100#Сколько приходится аминокислот на 1 ячейку
    print("T=", T)
    #if T>1:
    if T!=1:
        k=1
        t=T
        #print("t=", t)
        i=1
        j=1
        while i<l and j<100:
            a=min(k, t)
            proba[j]=round(proba[j]+a*list_hel_original[i],2)
            k=round(k-a,2)
            t=round(t-a,2)
            #print("a=",a," значение[",i,"]=", list_hel_original[i]," proba[", j,"]=", proba[j], " k=",k," t=",t, " i=", i, " j=",j)
            if k==0:
                i=i+1
                k=1
            if t==0:
                j=j+1
                t=T
            #print("k=",k, " t=", t)
        proba = [round(x/T, 2) for x in proba]
        #print("proba\n", proba)
    if T==1:
        proba=list_hel_original#просто копируем
    #if T<1:  
    #print("proba\n", proba)
    #print("proba\n", proba)
    return proba
    print('--------------------')



print('Добро пожаловать!')
start_time = time.perf_counter() #счётчик времени выполнения проги
        
df = Start()
Change_df (df)
Hist_maker(df)


print("Выберите какие структуры Вы хотели бы просмотреть: 1-альфа-спирали, 2-спирали 3_10, 3-бета-листы, 0-все приведённые и нерегулярные структуры.")
v = input()
print("Вы ввели: ", v)
if (v == "1"):
    print("Начинаю просматривать альфа-спирали")
    Alpha_helix(df)
else:
    if (v == "2"):
        print("Начинаю просматривать спирали 3-10") 
        ThreeTen_helix(df)
    else:
        if (v == "3"):
            print("Начинаю просматривать бета-листы")
            Beta_sheet(df)
        else: 
            if (v == "0"):
                print("Начинаю просматривать альфа-спирали")
                Alpha_helix(df)
                print("Начинаю просматривать спирали 3-10") 
                ThreeTen_helix(df)
                print("Начинаю просматривать бета-листы")
                Beta_sheet(df)
                print("Начинаю просматривать нерегулярные структуры")
                Irregular(df)
            else: print("Вы ввели неверное значение v. Попробуйте ещё раз")

#input()
#print(df.head)

df.to_excel("Result.xlsx")         
                   
print('Время работы скрипта: ',time.perf_counter()-start_time, 'сек')
#Время работы скрипта:  1739.8240654 сек // ~29минут

In [ ]:
"""
@author: ASUS

исходный файл: эксель со столбцами: 
    PDBid	chain	start*	end*
*-по PDB файлу

аут файл:


Описание функций:
    def Start - указываем путь к файлу xlsx, на выходе считанный датафрейм. [Вход: путь к экселю. Выход: считанный датафрейм]
    def Change_df - (!!! меняет исходный датафрейм) преобразует датафрейм к виду PDBid_chain PDBid chain start end length [Вход: датафрейм. Выход: преобразованный датафрейм. Сохраняет: эксель с преобразованным датафреймом]
    def Hist_maker - строит гистограмму по длинам подборки [Вход: подборка с длинами. Сохраняет: картинку гистограмму распределения длин молекул подборки]
    def Alpha_helix - добывает координаты альфа-спиралей [Вход: подборка наша. Вызывает: list_struc_maker (выход: датафрейм со спиралями их координатами)]
    def list_struc_maker - создаёт список длиной скоько а.о. и заполняет 1 там где есть структура [Вход: датафрейм со спиралями Helixes, длина белка l, начало белка start_domain. Вызывается в Alpha_helix]
    
"""

# Original code

In [12]:
import numpy as np
import pandas as pd
import time
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb

## Start

In [13]:
#_______________________________________________________

def Start ():
    print('-----START----')
    #-----Формируем первый датафрейм {PDBid chain start end}
    #print('Введите путь к файлу excel с его наименованием и .xlsx')
    #way = str(input())
    #'D:/Оля/курсовая/1_Обработка/1 Работа/проверяю расширенную обработку/pdbID.txt'
    path = 'book.xlsx'
    print("Вы ввели", path) 
    Podborka = pd.read_excel(path) #читвем датафрейм в переменную Podborka
    #print("Original Podborka\n", Podborka.head())
    #print('Типы:', Podborka.dtypes)
    #print('Колонки', Podborka.columns)
    #Podborka['PDBid']=str(Podborka['PDBid'])#преобразуем в строку
    #Podborka['chain']=Podborka['chain'].astype(str)#преобразуем в строку
    #Podborka['PDBid'].astype(str)
    #print('Типы:', Podborka.dtypes)
    display(Podborka.head(3))
    print('--------------------')
    return Podborka



# READY

## Change_df

In [14]:
def Change_df (Podborka):
    print('-----Change_df----')
    #-----Модифицируем датафрейм, добавляя length и PDBid_chain {PDBid_chain PDBid chain start end length}
    Podborka.insert(4, 'length', Podborka['end']-Podborka['start']+1) #считаем длину в а.о. каждого домена/цепи
    Podborka.insert(0, 'PDBid_Chain', Podborka['PDBid']+'_'+Podborka['chain'])#добавим идетификатор PDBid_Chain
    #print("Changed Podborka\n", Podborka.head())
    #Выведем саммари по этому датафрейму
    print('Саммари\n', Podborka.describe())  
    #Podborka.describe().to_csv("Podborka_describe.csv")
    Podborka.describe().to_excel("Podborka_describe.xlsx")#запишем себе данные дискрайба
    #и на всякий случай сохраним датафрейм
    Podborka.to_excel("Podborka.xlsx")
    #Заготовки для результатов по стурктурам
    Podborka["Alpha-helixes"] = np.nan
    Podborka["Beta-sheets"] = np.nan
    Podborka["ThreeTen-helixes"] = np.nan
    Podborka["Count_AlphaH"] = np.nan
    Podborka["Count_BetaS"] = np.nan
    Podborka["Count_ThreeTenH"] = np.nan
    display(Podborka.head(3))
    return Podborka
    print('--------------------')

## Hist_maker

In [15]:
def Hist_maker(Podborka):
    print('-----Hist_maker----')
    #Сразу распечатаем гистограмму (распределение длин белков подборки)
    ax = Podborka.hist(column='length', bins=25, grid=True, figsize=(12,8), color='royalblue', rwidth=0.9)  
    ax = ax[0]
    for x in ax:
        # Remove title
        x.set_title("Распределение длин белков подборки", size=20)
        # Set x-axis label
        x.set_xlabel("Длина, а.о.", labelpad=20, weight='bold', size=16)
        # Set y-axis label
        x.set_ylabel("Количество, структур", labelpad=20, weight='bold', size=16)
    fig = ax[0].get_figure()
    fig.savefig('Length distribution.jpg')#сохраним гистограмму
    print('--------------------')

## Alpha_helix

In [16]:
def Alpha_helix(Podborka):
    print('-----Alpha_helix----')
    g = 0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о. для PDB обычного
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о. для CATH
            #print("Beg_str=", Beg_str, "type=", type(Beg_str))
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о.
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "1"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_AlphaH"] = Helixes.shape[0]
                #input()
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет альфа-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "альфа-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "альфа-спиралей не обнаружено")
            Podborka.loc[i, "Count_AlphaH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Alpha-helixes"] = vse100
            #print("Podborka", Podborka
        display(Podborka.head(3))
        print('--------------------')

## ThreeTen_helix

In [17]:
def ThreeTen_helix(Podborka):
    print('-----ThreeTen_helix----')
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о.
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о. ЕСЛИ КАК В БД брать нумерацию
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "5"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_ThreeTenH"] = Helixes.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "3_10-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "3_10-спиралей не обнаружено")
            Podborka.loc[i, "Count_ThreeTenH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "ThreeTen-helixes"] = vse100
            #print("Podborka", Podborka)
    print('--------------------')    

## Beta_sheet

In [18]:
def Beta_sheet(Podborka):
    print('-----Beta_sheet----')
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_sheet_range"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_sht=pdbx_file [PDBid, "struct_sheet_range"]["sheet_id"]#id листа
            #print(Id_sht)
            Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_label_seq_id"]#начало листа а.о.
            #Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_seq_id"]#начало листа а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_label_seq_id"]#конец листа а.о.
            #End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_auth_seq_id"]#конец листа а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Chain_sht = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_asym_id"]#цепь https://www.biotite-python.org/examples/gallery/structure/sheet_arrangement.html
            #print("chain_sht", Chain_sht)
            #input()
            #ещё одна проверка если вдруг будет тупняк
            if Id_sht is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Sheeets = pd.DataFrame({'Id_sht' : Id_sht,'Beg_str' : Beg_str, 'End_str': End_str, 'Chain_sht': Chain_sht})
                #print('Sheeets\n', Sheeets)
                #print('type', type(Sheeets))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Sheeets = Sheeets[Sheeets.Chain_sht == chain_tut]#цепь нужная по Podborka
                Sheeets = Sheeets[Sheeets.Beg_str >= start_domain] #входит в домен
                Sheeets = Sheeets[Sheeets.End_str <= stop_domain] #входит в домен
                Sheeets.index = np.arange(len(Sheeets))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Sheeets.index",  Sheeets.index)
                #print("Sheeets которые подходят нам\n", Sheeets)
                Podborka.loc[i, "Count_BetaS"] = Sheeets.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Sheeets.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "бета-листов не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Sheeets, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_sht")
        else:
            print("В белке ", PDBid, "бета-листов не обнаружено")
            Podborka.loc[i, "Count_BetaS"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Beta-sheets"] = vse100
            #print("Podborka", Podborka)
    display(Podborka.head(3))
    print('--------------------')        


## Irregular

In [19]:
def Irregular(Podborka):
    print('-----Irregular----')
    for i, row in Podborka.iterrows():
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid просто для отслеживания
        #l = Podborka.iloc[i, 5]#выясняем актуальную длину
        #print("l=", l)
        #print("Podborka.iloc[i, 6]", Podborka.iloc[i, 6])
        list_alpha = Podborka.iloc[i, 6].split(";")#делаем список с данными об альфа
        list_beta = Podborka.iloc[i, 7].split(";")#делаем список с данными о бета-листах
        list_threeten = Podborka.iloc[i, 8].split(";")#делаем список с данными о 310-спиралях
        print("Данные: ", PDBid)
        #print("list_alpha=",list_alpha)
        proba = [0]*100#пустой для наших нерегулярных структур        
        for index in range(len(proba)):
            #print("index=", index, ", list_alpha[index]=", list_alpha[index])
            #print(type(l),"   ", type(float(list_alpha[index])))
            proba[index]=1-float(list_alpha[index])-float(list_beta[index])-float(list_threeten[index])
            proba[index] = round(proba[index], 2)
            if proba[index] < 0:
                print("значение в нерегулярных стурктурах <0!")
                print(PDBid, "index=", index, "proba=", proba)
                #input()
            #print("proba=", proba)
            #vse100 = ';'.join(str(proba))
            vse100 = (";".join(map(str,proba)))
            #print("vse100=", vse100)
            #vse100 = (";", join(map(str, proba)))
            Podborka.loc[i, "Irregular"] = vse100   
    print('--------------------') 

## list_struc_maker

In [20]:
def list_struc_maker(Helixes, l, start_domain):
    print('-----list_struc_maker----')
    #Считываем нужные альфа-спирали в список 
    list_hel_original=[0] * l#список, где будем выставлять наличие альфа-спиралей
    #заполняем его (это тот, что в аминокислотных остатках)
    for i in range(0,len(Helixes)):
        start = int(Helixes.iloc[i]["Beg_str"])
        stop = int(Helixes.iloc[i]["End_str"])
        #print("СТАРТ=", start, " СТОП=", stop)
        #ставим относительно начала и конца домена с учётом, что в списке начало с 0, а не 1 и учёт домена
        start = start-start_domain
        stop = stop - start_domain
        #print("start=", start, " stop=", stop)
        #вносим в нашу цепь данные
        for c in range (start, stop+1):
            list_hel_original[c]=1
            #print("list_hel_original", list_hel_original[c])
    #print("Итого по цепи")
    #for i in range(0, len(list_hel_original)):
        #print("list_hel_original[",i,"]=", list_hel_original[i])
    Itog = translater(list_hel_original, l)
    print("itog",Itog)
    return Itog
    print('--------------------')

## translater

In [21]:
def translater(list_hel_original, l):
    print('-----translater----')
    #делаем список из 100 ячеек (наши координаты, т.е. %)
    proba = [0]*100
    T=l/100#Сколько приходится аминокислот на 1 ячейку
    print("T=", T)
    #if T>1:
    if T!=1:
        k=1
        t=T
        #print("t=", t)
        i=1
        j=1
        while i<l and j<100:
            a=min(k, t)
            proba[j]=round(proba[j]+a*list_hel_original[i],2)
            k=round(k-a,2)
            t=round(t-a,2)
            #print("a=",a," значение[",i,"]=", list_hel_original[i]," proba[", j,"]=", proba[j], " k=",k," t=",t, " i=", i, " j=",j)
            if k==0:
                i=i+1
                k=1
            if t==0:
                j=j+1
                t=T
            #print("k=",k, " t=", t)
        proba = [round(x/T, 2) for x in proba]
        #print("proba\n", proba)
    if T==1:
        proba=list_hel_original#просто копируем
    #if T<1:  
    #print("proba\n", proba)
    print("proba\n", proba)
    return proba
    print('--------------------')


In [23]:
print('Добро пожаловать!')
start_time = time.perf_counter() #счётчик времени выполнения проги
        
df = Start()
Change_df (df)
#Hist_maker(df)

Добро пожаловать!
-----START----
Вы ввели book.xlsx


,PDBid,chain,start,end
0,1A3N,A,1,141
1,1A3N,B,2,146
2,1A4F,A,1,141


--------------------
-----Change_df----
Саммари
             start         end      length
count    6.000000    6.000000    6.000000
mean    34.333333  177.666667  144.333333
std     81.652108   82.502525    2.658320
min      0.000000  141.000000  141.000000
25%      1.000000  142.250000  142.000000
50%      1.000000  146.000000  145.500000
75%      1.750000  146.000000  146.000000
max    201.000000  346.000000  147.000000


,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,NaN,NaN,NaN,NaN,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,NaN,NaN,NaN,NaN,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,NaN,NaN,NaN,NaN,NaN,NaN


,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,NaN,NaN,NaN,NaN,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,NaN,NaN,NaN,NaN,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,NaN,NaN,NaN,NaN,NaN,NaN
3,1A4F_B,1A4F,B,1,146,146,NaN,NaN,NaN,NaN,NaN,NaN
4,1ASH_A,1ASH,A,0,146,147,NaN,NaN,NaN,NaN,NaN,NaN
5,1IT2_B,1IT2,B,201,346,146,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print("Выберите какие структуры Вы хотели бы просмотреть: 1-альфа-спирали, 2-спирали 3_10, 3-бета-листы, 0-все приведённые и нерегулярные структуры.")
v = input()
print("Вы ввели: ", v)
if (v == "1"):
    print("Начинаю просматривать альфа-спирали")
    Alpha_helix(df)
else:
    if (v == "2"):
        print("Начинаю просматривать спирали 3-10") 
        ThreeTen_helix(df)
    else:
        if (v == "3"):
            print("Начинаю просматривать бета-листы")
            Beta_sheet(df)
        else: 
            if (v == "0"):
                print("Начинаю просматривать альфа-спирали")
                Alpha_helix(df)
                print("Начинаю просматривать спирали 3-10") 
                ThreeTen_helix(df)
                print("Начинаю просматривать бета-листы")
                Beta_sheet(df)
                print("Начинаю просматривать нерегулярные структуры")
                Irregular(df)
            else: print("Вы ввели неверное значение v. Попробуйте ещё раз")


Выберите какие структуры Вы хотели бы просмотреть: 1-альфа-спирали, 2-спирали 3_10, 3-бета-листы, 0-все приведённые и нерегулярные структуры.


 1


Вы ввели:  1
Начинаю просматривать альфа-спирали
-----Alpha_helix----
Данные:  1A3N 141 A 1 141 g= 0
-----list_struc_maker----
-----translater----
T= 1.41
proba
 [0.0, 0.0, 0.58, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.83, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.65, 0.0, 0.0, 0.52, 1.0, 1.0, 0.32, 0.97, 1.0, 1.0, 1.0, 1.0, 1.0, 0.41, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.45, 0.0, 0.0]
itog [0.0, 0.0, 0.58, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.83, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.65, 0.0, 0.0, 0.52, 1.0, 1.0, 0.32, 0.97, 1.0, 1.0, 1.0, 1.0, 1.0,

/tmp/ipykernel_79/3399659204.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0.11;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.83;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0.65;0.0;0.0;0.52;1.0;1.0;0.32;0.97;1.0;1.0;1.0;1.0;1.0;0.41;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.02;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0.45;0.0;0.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Podborka.loc[i, "Alpha-helixes"] = vse100


,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,NaN,NaN,NaN,NaN,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,NaN,NaN,NaN,NaN,NaN,NaN


...

--------------------
Данные:  1ASH 147 A 0 146 g= 4
-----list_struc_maker----
-----translater----
T= 1.47
proba
 [0.0, 0.32, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.52, 0.0, 0.0, 0.0, 0.39, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.63, 1.0, 1.0, 1.0, 0.46, 0.86, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.06, 0.0, 0.22, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.03, 0.0, 0.0, 0.57, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.67, 0.0, 0.0, 0.61, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.64]
itog [0.0, 0.32, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.52, 0.0, 0.0, 0.0, 0.39, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.63, 1.0, 1.0, 1.0, 0.46, 0.86, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.06, 0.0, 0.22, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.03, 0.0, 0

,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,0.0;0.0;0.62;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0...,NaN,NaN,6.0,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN


--------------------
Данные:  1IT2 146 B 201 346 g= 5
В белке  1IT2 альфа-спиралей не обнаружено


,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,0.0;0.0;0.62;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0...,NaN,NaN,6.0,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN


--------------------


In [25]:


#input()
#print(df.head)

#df.to_excel("Result.xlsx")         
                   
print('Время работы скрипта: ',time.perf_counter()-start_time, 'сек')
#Время работы скрипта:  1739.8240654 сек // ~29минут

Время работы скрипта:  107.17769930000009 сек


In [26]:
df

,PDBid_Chain,PDBid,chain,start,end,length,Alpha-helixes,Beta-sheets,ThreeTen-helixes,Count_AlphaH,Count_BetaS,Count_ThreeTenH
0,1A3N_A,1A3N,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN
1,1A3N_B,1A3N,B,2,146,145,0.0;0.0;0.62;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0...,NaN,NaN,6.0,NaN,NaN
2,1A4F_A,1A4F,A,1,141,141,0.0;0.0;0.58;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1...,NaN,NaN,5.0,NaN,NaN
3,1A4F_B,1A4F,B,1,146,146,0.0;0.0;0.0;0.95;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0...,NaN,NaN,7.0,NaN,NaN
4,1ASH_A,1ASH,A,0,146,147,0.0;0.32;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;0.52;...,NaN,NaN,7.0,NaN,NaN
5,1IT2_B,1IT2,B,201,346,146,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,NaN,NaN,0.0,NaN,NaN


# polish code

In [2]:
#import numpy as np
import pandas as pd
#import time
#import biotite.structure.io.pdbx as pdbx
#import biotite.database.rcsb as rcsb

## Start --> read_prot_set

In [3]:
#TODO: check up the mode asserts?
# is exist the path/file

#START in previous life

def read_prot_set(path: str, mode: str = 'xlsx'):
    """
    Read protein set data from file through the path and return a pandas dataframe. The following data structure is required: PDBid, chain, start, end. Header is needed.
    arguments:
        - path (str): path to the file for reading
        - mode (str): file format to read. Only xlsx, csv and tsv formats are allowed
    return:
        - prot_set_df (pandas.DataFrame): dataframe of the protein set
    """
    print('----read_prot_set----')

    match mode:
        case 'xlsx':
            prot_set_df = pd.read_excel(path)
        case 'csv':
            prot_set_df = pd.read_csv(path)
        case 'tsv':
            prot_set_df = pd.read_csv(path, sep='\t')

    prot_set_df.insert(0, 'PDBid_Chain', prot_set_df['PDBid']+'_'+prot_set_df['chain'])#add unique identificator for each chain -> PDBid_Chain

    return prot_set_df

In [4]:
df = read_prot_set('book.xlsx')
df

----read_prot_set----


,PDBid_Chain,PDBid,chain,start,end
0,1A3N_A,1A3N,A,1,141
1,1A3N_B,1A3N,B,2,146
2,1A4F_A,1A4F,A,1,141
3,1A4F_B,1A4F,B,1,146
4,1ASH_A,1ASH,A,0,146
5,1IT2_B,1IT2,B,201,346


In [5]:
read_prot_set('book.csv', mode='csv')

----read_prot_set----


,PDBid_Chain,PDBid,chain,start,end
0,1A3N_A,1A3N,A,1,141
1,1A3N_B,1A3N,B,2,146
2,1A4F_A,1A4F,A,1,141
3,1A4F_B,1A4F,B,1,146
4,1ASH_A,1ASH,A,0,146
5,1IT2_B,1IT2,B,201,346


In [6]:
read_prot_set('book.tsv', mode='tsv')

----read_prot_set----


,PDBid_Chain,PDBid,chain,start,end
0,1A3N_A,1A3N,A,1,141
1,1A3N_B,1A3N,B,2,146
2,1A4F_A,1A4F,A,1,141
3,1A4F_B,1A4F,B,1,146
4,1ASH_A,1ASH,A,0,146
5,1IT2_B,1IT2,B,201,346


## Change_df --> count_length

In [7]:
#TODO: is it necessary to save this step. Or maiby only with special mode

#Change_df in previous life


def calculate_length(prot_set_df):
    """
    Calculate lengths for each chain in amino acids. Add an additional column to dataframe of the protein set.
    arguments:
        - prot_set_df (pandas.DataFrame): dataframe of the protein set
    """
    print('----calculate_length----')

    prot_set_df['length'] = prot_set_df['end'] - prot_set_df['start'] + 1

    #is it n=necessary?
    #print('Саммари\n', prot_set_df.describe())
    #prot_set_df.describe().to_excel("Podborka_describe.xlsx")#запишем себе данные дискрайба
    #prot_set_df.to_excel("Podborka.xlsx")

    return prot_set_df#это надо?

In [8]:
calculate_length(df)

----calculate_length----


,PDBid_Chain,PDBid,chain,start,end,length
0,1A3N_A,1A3N,A,1,141,141
1,1A3N_B,1A3N,B,2,146,145
2,1A4F_A,1A4F,A,1,141,141
3,1A4F_B,1A4F,B,1,146,146
4,1ASH_A,1ASH,A,0,146,147
5,1IT2_B,1IT2,B,201,346,146


## Hist_maker --> ∅

## Alpha_helix | ThreeTen_helix | Beta_sheet | Irregular --> OOP

In [9]:
#dataclass SecondaryStructure
#Helix(SecondaryStructure)
#AlphaHelix(Helix)
#ThreeTenHelix(Helix)
#BetaSheet(SecondaryStructure)
#Irregular(SecondaryStructure)

#let it be node for which structure schould be processed 
#let it be mode for separate or full result for each type of secondary structure (but if full then join it on the last step)
